In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import itertools
from imblearn.over_sampling import SMOTE

In [4]:
data = pd.read_csv('data_new.csv')
pd.options.display.max_rows = 100
data

,X.60,X.59,X.58,X.57,X.56,X.55,X.54,X.53,X.52,X.51,...,days_employed,has_mobile_phone,has_work_phone,has_phone,has_email,occupation_type,num_fam_members,observed_duration,default,time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4542,1,2,1,1,1,2,16,0,16
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4542,1,2,1,1,1,2,15,0,15
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1134,1,1,1,1,18,2,30,0,30
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3051,1,1,2,2,16,1,5,0,5
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3051,1,1,2,2,16,1,5,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1739,1,1,1,1,5,3,18,0,18
33440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1739,1,1,1,1,5,3,18,0,18
33441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1739,1,1,1,1,5,3,13,0,13
33442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1739,1,1,1,1,5,3,2,0,2


In [5]:
data.columns

Index(['X.60', 'X.59', 'X.58', 'X.57', 'X.56', 'X.55', 'X.54', 'X.53', 'X.52',
       'X.51', 'X.50', 'X.49', 'X.48', 'X.47', 'X.46', 'X.45', 'X.44', 'X.43',
       'X.42', 'X.41', 'X.40', 'X.39', 'X.38', 'X.37', 'X.36', 'X.35', 'X.34',
       'X.33', 'X.32', 'X.31', 'X.30', 'X.29', 'X.28', 'X.27', 'X.26', 'X.25',
       'X.24', 'X.23', 'X.22', 'X.21', 'X.20', 'X.19', 'X.18', 'X.17', 'X.16',
       'X.15', 'X.14', 'X.13', 'X.12', 'X.11', 'X.10', 'X.9', 'X.8', 'X.7',
       'X.6', 'X.5', 'X.4', 'X.3', 'X.2', 'X.1', 'X0', 'sex', 'has_car',
       'has_property', 'num_children', 'annual_income', 'income_type',
       'education_type', 'marital_status', 'housing_type', 'age',
       'days_employed', 'has_mobile_phone', 'has_work_phone', 'has_phone',
       'has_email', 'occupation_type', 'num_fam_members', 'observed_duration',
       'default', 'time'],
      dtype='object')

In [6]:
# #data = data[['sex', 'has_car',
#        'has_property', 'num_children', 'annual_income', 'income_type',
#        'education_type', 'marital_status', 'housing_type', 'age',
#        'days_employed', 'has_mobile_phone', 'has_work_phone', 'has_phone',
#        'has_email', 'occupation_type', 'num_fam_members',
#        'default']]

In [7]:
# changing which columns to predict with
data = data[['sex', 'has_car',
       'has_property', 'num_children', 'annual_income', 'income_type',
       'education_type', 'marital_status', 'housing_type', 'age',
       'days_employed', 'has_mobile_phone', 'has_work_phone', 'has_phone',
       'has_email', 'occupation_type', 'num_fam_members',
       'default']]
data.shape

(33444, 18)

In [8]:
df_majority = data[data['default']==0]
df_minority = data[data['default']==1]

In [9]:
df_majority.shape

(32846, 18)

In [10]:
df_minority_unsampled = resample(df_minority,
                                 replace = True, # sample with replacement
                                 n_samples = 32846, # match majority class
                                 random_state = 42) #reproducable results

df_unsampled = pd.concat([df_minority_unsampled, df_majority])

In [11]:
df_unsampled.shape

(65692, 18)

In [12]:
Y = df_unsampled['default']
X = df_unsampled.loc[:, df_unsampled.columns != 'default']

In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
X.shape

(65692, 17)

In [14]:
# using feature selection with logistic regression to find the 5 most significant features (very crude method)
selector = SelectKBest(f_classif, k=5)
X = selector.fit_transform(X, Y)
selector.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [11] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


array(['sex', 'has_property', 'marital_status', 'days_employed',
       'occupation_type'], dtype=object)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    random_state = 42,
                                                    stratify=Y)

In [16]:
Y_train

10814    1
4547     1
22593    0
22281    0
29488    1
        ..
21431    1
23773    1
14540    0
1149     1
16328    1
Name: default, Length: 49269, dtype: int64

In [21]:
model1 = LogisticRegression(C=0.8,
                           random_state=0,
                           solver='lbfgs')
model1.fit(X_train, Y_train)
Y_predict = model1.predict(X_test)

print('The Accuracy Score for Logistic Regression is {:.5}'.format(accuracy_score(Y_test, Y_predict)))
print('The F1 Score for Logistic Regression is {:.5}'.format(f1_score(Y_test, Y_predict)))
print('The Confusion Matrix for Logistic Regression is {}'.format(confusion_matrix(Y_test, Y_predict)))

The Accuracy Score for Logistic Regression is 0.56177
The F1 Score for Logistic Regression is 0.5342
The Confusion Matrix for Logistic Regression is [[5099 3113]
 [4084 4127]]
